# FM를 이용한 추천시스템2
- user, item
- 다른 컬럼 특성 추가 : Occupation, Gender, Genre

## Data load

In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

In [2]:
# Load the u.user file into a dataframe
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.user', sep='|', names=u_cols, encoding='latin-1')

In [3]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
# Load the u.item file into a dataframe / 해당 데이터프레임은 원핫으로 이루어져 있다.
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL',
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.item', sep='|', names=i_cols, encoding='latin-1')


In [5]:
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
# Load the u.data file into a dataframe
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


## user encoding

In [7]:
user_dict = {}
for i in set(ratings['user_id']):
    user_dict[i] = len(user_dict)
n_user = len(user_dict)
n_user

943

## Item encoding

In [8]:
item_dict = {}
start_point = n_user
for i in set(ratings['movie_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)
start_point += n_item
num_x = start_point               # Total number of x
ratings = shuffle(ratings, random_state=1)

In [9]:
item_dict.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

In [10]:
item_dict.values()

dict_values([943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 

## Occupation encoding

In [11]:
occ_dict = {}
for i in set(users['occupation']):
    occ_dict[i] = start_point + len(occ_dict)
n_occ = len(occ_dict)
start_point += n_occ


In [12]:
occ_dict

{'doctor': 2625,
 'entertainment': 2626,
 'technician': 2627,
 'other': 2628,
 'engineer': 2629,
 'marketing': 2630,
 'healthcare': 2631,
 'retired': 2632,
 'programmer': 2633,
 'lawyer': 2634,
 'scientist': 2635,
 'librarian': 2636,
 'administrator': 2637,
 'educator': 2638,
 'writer': 2639,
 'none': 2640,
 'homemaker': 2641,
 'executive': 2642,
 'student': 2643,
 'artist': 2644,
 'salesman': 2645}

## Gender encoding

In [13]:
gender_dict = {}
for i in set(users['sex']):
    gender_dict[i] = start_point + len(gender_dict)
n_gender = len(gender_dict)
start_point += n_gender


In [14]:
gender_dict

{'F': 2646, 'M': 2647}

## Genre encoding

In [15]:
genre_dict = {}
genre = ['unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

for i in genre:
    genre_dict[i] = start_point + len(genre_dict)
n_genre = len(genre_dict)
start_point += n_genre
age_index = start_point
start_point += 1
num_x = start_point             # Total number of x


In [16]:
genre_dict

{'unknown': 2648,
 'Action': 2649,
 'Adventure': 2650,
 'Animation': 2651,
 "Children's": 2652,
 'Comedy': 2653,
 'Crime': 2654,
 'Documentary': 2655,
 'Drama': 2656,
 'Fantasy': 2657,
 'Film-Noir': 2658,
 'Horror': 2659,
 'Musical': 2660,
 'Mystery': 2661,
 'Romance': 2662,
 'Sci-Fi': 2663,
 'Thriller': 2664,
 'War': 2665,
 'Western': 2666}

## Merge data

In [ ]:
movies = movies.drop(['title', 'release date', 'video release date', 'IMDB URL'], axis=1) 
users = users.drop(['zip_code'], axis=1)
ratings = ratings.drop(['timestamp'], axis=1)

x = pd.merge(ratings, movies, how='outer', on='movie_id')
x = pd.merge(x, users, how='outer', on='user_id')
x = shuffle(x, random_state=1) # 행 단위로 무작위 섞음
x.head()

,user_id,movie_id,rating,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,age,sex,occupation
43660,396,237,4,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,57,M,engineer
87278,823,197,5,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,27,M,artist
14317,144,198,4,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,53,M,programmer
81932,760,682,3,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,35,F,other
95321,894,1462,3,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,47,M,educator


## Generate X data

In [ ]:
data = []
y = []
age_mean = np.mean(x['age'])
age_std = np.std(x['age'])
w0 = np.mean(x['rating'])

for i in range(len(x)):
    case = x.iloc[i]
    x_index = []
    x_value = []
    x_index.append(user_dict[case['user_id']])     # User id encoding
    x_value.append(1)                              # 각 특징이 존재한다는 것의 의미 
    x_index.append(item_dict[case['movie_id']])    # Movie id encoding
    x_value.append(1)
    x_index.append(occ_dict[case['occupation']])   # Occupation id encoding
    x_value.append(1)
    x_index.append(gender_dict[case['sex']])       # Gender id encoding
    x_value.append(1)
    
    for j in genre:
        if case[j] == 1:               # 해당 장르가 1
            x_index.append(genre_dict[j])
            x_value.append(1)
    x_index.append(age_index)
    x_value.append((case['age'] - age_mean) / age_std)
    
    data.append([x_index, x_value])
    y.append(case['rating'] - w0) # 평점 - 평균 평점 , 이는 모델이 편향을 학습할 수 있도록 돕는다.
    
    if (i % 10000) == 0:
        print('Encoding ', i, ' cases...')

Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...
Encoding  40000  cases...
Encoding  50000  cases...
Encoding  60000  cases...
Encoding  70000  cases...
Encoding  80000  cases...
Encoding  90000  cases...


In [20]:
data[:5]

[[[395, 1179, 2629, 2647, 2656, 2662, 2667],
  [1, 1, 1, 1, 1, 1, np.float64(2.0782714698386564)]],
 [[822, 1139, 2644, 2647, 2656, 2662, 2667],
  [1, 1, 1, 1, 1, 1, np.float64(-0.5163084264649329)]],
 [[143, 1140, 2633, 2647, 2664, 2667],
  [1, 1, 1, 1, 1, np.float64(1.7323274836648446)]],
 [[759, 1624, 2628, 2646, 2659, 2661, 2664, 2667],
  [1, 1, 1, 1, 1, 1, 1, np.float64(0.17557954588269098)]],
 [[893, 2404, 2638, 2647, 2654, 2656, 2662, 2667],
  [1, 1, 1, 1, 1, 1, 1, np.float64(1.2134115044041267)]]]

- data 리스트 : FM모델 학습을 위해 전처리된 특징 인덱스와 값을 의미 (= 입력 특성)
    - [사용자 ID 인덱스, 아이템 ID 인덱스, 직업 ID 인덱스, 성별 ID 인덱스, 장르 ID 인덱스, 나이 특성의 인덱스]
    - x_index : 각 특징을 숫자 인덱스로 변환한 값
    - x_value : 각 특징을 one-hot 인코딩한 값

In [26]:
x_index

[929, 1079, 2635, 2646, 2656, 2667]

In [27]:
x_value

[1, 1, 1, 1, 1, np.float64(-0.4298224299214799)]

In [24]:
y[:5]

[np.float64(0.4701399999999998),
 np.float64(1.4701399999999998),
 np.float64(0.4701399999999998),
 np.float64(-0.5298600000000002),
 np.float64(-0.5298600000000002)]

## RMSE

In [28]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

## FM class

In [ ]:
class FM():
    def __init__(self, N, K, data, y, alpha, beta, train_ratio=0.75, iterations=100, tolerance=0.005, l2_reg=True, verbose=True):
        self.K = K                  # Number of latent factors
        self.N = N                  # Number of x (variables)
        self.n_cases = len(data)    # N of observations
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.l2_reg = l2_reg
        self.tolerance = tolerance
        self.verbose = verbose
        
        # w 초기화
        self.w = np.random.normal(scale=1./self.N, size=(self.N))
        # v 초기화
        self.v = np.random.normal(scale=1./self.K, size=(self.N, self.K))
        # Train/Test 분리
        cutoff = int(train_ratio * len(data))
        self.train_x = data[:cutoff]
        self.test_x = data[cutoff:]
        self.train_y = y[:cutoff]
        self.test_y = y[cutoff:]

    def test(self):                                     # Training 하면서 RMSE 계산
        
        # SGD를 iterations 숫자만큼 수행
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        for i in range(self.iterations):
            rmse1 = self.sgd(self.train_x, self.train_y)        # SGD & Train RMSE 계산
            rmse2 = self.test_rmse(self.test_x, self.test_y)    # Test RMSE 계산
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                       # New best record
                best_RMSE = rmse2
                best_iteration = i
            elif (rmse2 - best_RMSE) > self.tolerance:  # RMSE is increasing over tolerance
                break
        print(best_iteration, best_RMSE) 
        return training_process

    # w, v 업데이트를 위한 Stochastic gradient descent
    def sgd(self, x_data, y_data):
        y_pred = []
        for data, y in zip(x_data, y_data):
            x_idx = data[0]
            x_0 = np.array(data[1])     # xi axis=0 [1, 2, 3]
            x_1 = x_0.reshape(-1, 1)    # xi axis=1 [[1], [2], [3]]

            # biases
            bias_score = np.sum(self.w[x_idx] * x_0)

            # score 계산
            vx = self.v[x_idx] * (x_1)          # v matrix * x
            sum_vx = np.sum(vx, axis=0)         # sigma(vx)
            sum_vx_2 = np.sum(vx * vx, axis=0)  # ( v matrix * x )의 제곱
            latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

            # 예측값 계산
            y_hat = bias_score + latent_score
            y_pred.append(y_hat)
            error = y - y_hat
            # w, v 업데이트
            if self.l2_reg:     # regularization이 있는 경우
                self.w[x_idx] += error * self.alpha * (x_0 - self.beta * self.w[x_idx])
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1) - self.beta * self.v[x_idx])
            else:               # regularization이 없는 경우
                self.w[x_idx] += error * self.alpha * x_0
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1))
        return RMSE(y_data, y_pred)

    def test_rmse(self, x_data, y_data):
        y_pred = []
        for data , y in zip(x_data, y_data):
            y_hat = self.predict(data[0], data[1])
            y_pred.append(y_hat)
        return RMSE(y_data, y_pred)

    def predict(self, idx, x):
        x_0 = np.array(x)
        x_1 = x_0.reshape(-1, 1)

        # biases
        bias_score = np.sum(self.w[idx] * x_0)

        # score 계산
        vx = self.v[idx] * (x_1)
        sum_vx = np.sum(vx, axis=0)
        sum_vx_2 = np.sum(vx * vx, axis=0)
        latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2) # square : 각 원소 제곱

        # 예측값 계산
        y_hat = bias_score + latent_score
        return y_hat


- `sum_vx` :  $\sum(v_i \cdot x_i)$, 잠재 벡터 v와 입력 특성x의 곱의 합
- `np.square(sum_vx)` : 합을 제곱한 값, $ \left( \sum v_i \cdot x_i \right)^2 $

In [45]:
K = 200
fm1 = FM(num_x, K, data, y, alpha=0.00005, beta=0.002, train_ratio=0.75, iterations=900, tolerance=0.0001, l2_reg=True, verbose=True)
result = fm1.test()

Iteration: 10 ; Train RMSE = 1.078525 ; Test RMSE = 1.077623
Iteration: 20 ; Train RMSE = 1.054438 ; Test RMSE = 1.054641
Iteration: 30 ; Train RMSE = 1.028518 ; Test RMSE = 1.029895
Iteration: 40 ; Train RMSE = 1.001467 ; Test RMSE = 1.004820
Iteration: 50 ; Train RMSE = 0.977142 ; Test RMSE = 0.983491
Iteration: 60 ; Train RMSE = 0.958183 ; Test RMSE = 0.968144
Iteration: 70 ; Train RMSE = 0.944274 ; Test RMSE = 0.957876
Iteration: 80 ; Train RMSE = 0.933922 ; Test RMSE = 0.950908
Iteration: 90 ; Train RMSE = 0.925977 ; Test RMSE = 0.946031
Iteration: 100 ; Train RMSE = 0.919703 ; Test RMSE = 0.942541
Iteration: 110 ; Train RMSE = 0.914614 ; Test RMSE = 0.939997
Iteration: 120 ; Train RMSE = 0.910373 ; Test RMSE = 0.938101
Iteration: 130 ; Train RMSE = 0.906737 ; Test RMSE = 0.936649
Iteration: 140 ; Train RMSE = 0.903537 ; Test RMSE = 0.935500
Iteration: 150 ; Train RMSE = 0.900652 ; Test RMSE = 0.934563
Iteration: 160 ; Train RMSE = 0.897998 ; Test RMSE = 0.933774
Iteration: 170 ; 

1.	`RMSE` : 모델의 성능 평가를 위한 손실 함수.
    - $RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2}$

2.	`Bias Term` : 유저/아이템 등 각 피처의 1차 가중치와 값의 곱의 합.
    - $\text{bias\_score} = \sum_{i=1}^n w_i \cdot x_i$
    - 코드 수식 : $\sum{i \in \text{x\_idx}} w_i \cdot x_i$
3.	`Latent Term` : 잠재 요인을 사용한 2차 상호작용 계산.
    - $\text{latent\_score} = \sum_{i=1}^n \sum_{j=i+1}^n \langle v_i, v_j \rangle \cdot x_i \cdot x_j$
    - $\text{latent\_score} = \frac{1}{2} \sum_{f=1}^K \left( \left( \sum_{i \in \text{x\_idx}} v_{i, f} \cdot x_i \right)^2 - \sum_{i \in \text{x\_idx}} \left( v_{i, f} \cdot x_i \right)^2 \right)$
4.	`예측값` : Bias Term과 Latent Term의 합.
    - $\hat{y} = \text{bias\_score} + \text{latent\_score}$
5.	`가중치 업데이트` : SGD 기반의 1차 및 2차 가중치 업데이트.
    - 1차 가중치 w업데이트(코드 수식) : $w_i = w_i + \alpha \cdot \text{error} \cdot (x_i - \beta \cdot w_i)$
    - 잠재요인 v 업데이트(코드 수식) : $v_{i, f} = v_{i, f} + \alpha \cdot \text{error} \cdot \left( x_i \cdot \sum_{j \in \text{x\_idx}} v_{j, f} \cdot x_j - v_{i, f} \cdot x_i^2 - \beta \cdot v_{i, f} \right)$
6.	`정규화` : 연속형 변수의 표준화.
    - $x_i = \frac{x_i - \mu}{\sigma}$